In [1]:
import dimod, numpy
import networkx

done = False
choice = input("Choose problem to solve: \n(M) Max Cut\n(T) Traveling Sales Man\n(R) Room assignment\nEnter letter: ")
choice = choice.upper()

def SolveMAXCUT():
    NoNodes = 4
    h = {}
    j = {}
    y = 0
    for i in range(NoNodes-1):
        y = i+1
        while y<NoNodes:
            if (y+1)<=NoNodes:
                cost = numpy.random.uniform(-1, 1)
                j[(i,y)] = cost
                y +=1
    BQM = dimod.BinaryQuadraticModel(h,j,0.0, dimod.BINARY) 
    computer = dimod.ExactSolver()
    answer = computer.sample(BQM)
    LowestCost = answer.first.energy
    solution = answer.first.sample
    print(f"Solution:", solution)
    print(f"Its cost:", LowestCost)

def SolveTSP():
    NoCities = 4
    h = {} 
    j = {}   
    offset = 0.0

    # Initializing linear terms to 0 for city and position variables
    for i in range(NoCities):
        for p in range(NoCities):
            h[(i, p)] = 0.0

    # Random distances
    distances = {}
    for i in range(NoCities):
        for y in range(i+1, NoCities):
            d = numpy.random.uniform(1, 10)
            distances[(i, y)] = d
            distances[(y, i)] = d

    A = 10.0

    # First Constraint: each city i appears once
    for i in range(NoCities):
        for p in range(NoCities):
            h[(i, p)] += -A
        for w in range(NoCities):
            for k in range(w+1, NoCities):
                j[((i, w), (i, k))] = j.get(((i, w), (i, k)), 0.0) + 2.0 * A
        offset += A  # constant term (optional)

    # Second Constraint each position p has one city
    for p in range(NoCities):
        for i in range(NoCities):
            h[(i, p)] += -A
        for k in range(NoCities):
            for w in range(k+1, NoCities):
                j[((k, p), (w, p))] = j.get(((k, p), (w, p)), 0.0) + 2.0 * A
        offset += A

    # Add travel costs: for consecutive positions
    for p in range(NoCities - 1):
        for i in range(NoCities):
            for city in range(NoCities):
                if i == city:
                    continue
                cost = distances[(i, city)]
                y = ((i, p), (city, p+1))
                j[y] = j.get(y, 0.0) + cost

    BQM = dimod.BinaryQuadraticModel(h, j, offset, dimod.BINARY)
    solver = dimod.ExactSolver()
    result = solver.sample(BQM)

    best = result.first
    print("Energy (best):", best.energy)
    print("Binary solution (best):", best.sample)

    route = [None] * NoCities
    for (city, pos), val in best.sample.items():
        if val == 1:
            route[pos] = city
    print("Best route:", route)


def SolveRooms():
    N_rooms = 4
    N_classes = 4
    # set data
    chairs = [30, 40, 20, 50]     
    students = [25, 35, 15, 45]   
    A = 20.0
    h = {}
    j = {}
    offset = 0.0

    # Create variable for each (class, room)
    for c in range(N_classes):
        for r in range(N_rooms):
            h[(c, r)] = 0.0

    # Minimizing wasted chairs
    for c in range(N_classes):
        for r in range(N_rooms):
            waste = (chairs[r] - students[c])**2
            h[(c, r)] += waste

    # Constraint 1: each class assigned to one room
    for c in range(N_classes):
        for r in range(N_rooms):
            h[(c, r)] += -A  # linear part
        for k in range(N_rooms):
            for r2 in range(k+1, N_rooms):
                j[((c, k), (c, r2))] = j.get(((c, k), (c, r2)), 0.0) + 2*A
        offset += A

    # Constraint 2: each room has one class
    for r in range(N_rooms):
        for c in range(N_classes):
            h[(c, r)] += -A
        for c1 in range(N_classes):
            for c2 in range(c1+1, N_classes):
                j[((c1, r), (c2, r))] = j.get(((c1, r), (c2, r)), 0.0) + 2*A
        offset += A

    BQM = dimod.BinaryQuadraticModel(h, j, offset, dimod.BINARY)
    solver = dimod.ExactSolver()
    result = solver.sample(BQM)
    best = result.first

    print("Lowest energy:", best.energy)
    print("Binary assignment:", best.sample)


    for c in range(N_classes):
        for r in range(N_rooms):
            if best.sample[(c, r)] == 1:
                print(f"Class {c+1} assigned to Room {r+1}")

while done==False:
    if choice == "M":
        done = True
        SolveMAXCUT()
        
    elif choice == "T":
        done = True
        SolveTSP()
    elif choice == "R":
        done = True
        SolveRooms()
    else:
        choice = input("Choose a valid option")


Choose problem to solve: 
(M) Max Cut
(T) Traveling Sales Man
(R) Room assignment
Enter letter:  t


Energy (best): 10.18945600148917
Binary solution (best): {(0, 0): np.int8(0), (0, 1): np.int8(0), (0, 2): np.int8(1), (0, 3): np.int8(0), (1, 0): np.int8(0), (1, 1): np.int8(0), (1, 2): np.int8(0), (1, 3): np.int8(1), (2, 0): np.int8(1), (2, 1): np.int8(0), (2, 2): np.int8(0), (2, 3): np.int8(0), (3, 0): np.int8(0), (3, 1): np.int8(1), (3, 2): np.int8(0), (3, 3): np.int8(0)}
Best route: [2, 3, 0, 1]
